In [1]:
import keras
from tensorflow.python.client import device_lib

#print(device_lib.list_local_devices())
import numpy as np
import pandas as pd
import re


import os

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding, Activation
from keras.layers import Dense, Input, Flatten, Bidirectional
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model

from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.layers import Dense, Embedding, LSTM, GRU

MAX_SEQUENCE_LENGTH = 2000
MAX_NB_WORDS = 600000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
print("Import Successful")

TypeError: __init__() got an unexpected keyword argument 'metaclass'

In [2]:
def clean_str(string):
    """
    Cleaning of dataset
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()


In [3]:
# Input Data preprocessing
data_train = pd.read_csv('data/train_Mixed.csv')
#print(data_train.columns)
#print('What the raw input data looks like:')
#print(data_train[0:5])
texts = []
labels = []

for i in range(data_train.text.shape[0]):
    text1 = data_train.title[i]
    text2 = data_train.text[i]
    text = str(text1) +""+ str(text2)
    texts.append(text)
    labels.append(data_train.label[i])
    
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print("Data Improted")
#print('Found %s unique tokens.' % len(word_index))

In [4]:
# Pad input sequences
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels),num_classes = 2)
# print('Shape of data tensor:', data.shape)
# print('Shape of label tensor:', labels.shape)
print("preprocess complete")

preprocess complete


In [5]:
# Train test validation Split
from sklearn.model_selection import train_test_split

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
x_train, x_test, y_train, y_test = train_test_split( data, labels, test_size=0.20, random_state=42)
x_test, x_val, y_test, y_val = train_test_split( x_test, y_test, test_size=0.50, random_state=42)
# print('Size of train, validation, test:', len(y_train), len(y_val), len(y_test))

# print('real & fake news in train,valt,test:')
# print(y_train.sum(axis=0))
# print(y_val.sum(axis=0))
# print(y_test.sum(axis=0))
print("train_test_split_complete")

train_test_split_complete


In [6]:
#Using Pre-trained word embeddings
GLOVE_DIR = "data" 
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    #print(values[1:])
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

#print('Total %s word vectors in Glove.' % len(embeddings_index))

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH)

print("Glove Complete")

Glove Complete


In [7]:
modell = Sequential()
modell.add(embedding_layer)
for i in range(0,2):
    modell.add(Conv1D(filters=1024, kernel_size=1, padding='same', activation='relu'))
    modell.add(BatchNormalization())

for i in range(0,5):
    modell.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
    modell.add(BatchNormalization())
    modell.add(Activation('relu'))
    
for i in range(0,5):
    modell.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
    modell.add(BatchNormalization())
    modell.add(Activation('relu'))
    
for i in range(0,3):
    modell.add(Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'))
    modell.add(BatchNormalization())
    modell.add(Activation('relu'))
    modell.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
    modell.add(BatchNormalization())
    modell.add(MaxPooling1D(pool_size=2))
    modell.add(Activation('relu'))
    
for i in range (0,7):
    modell.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    modell.add(BatchNormalization())
    modell.add(Activation('relu'))
for i in range (0,5):
    modell.add(Conv1D(filters=256, kernel_size=3, padding='same', activation='relu'))
    modell.add(BatchNormalization())
    modell.add(Activation('relu'))
    
for i in range (0,3):
    modell.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    modell.add(BatchNormalization())
    
for i in range (0,5):
    modell.add(Conv1D(filters=512, kernel_size=3, padding='same', activation='relu'))
    modell.add(BatchNormalization())
    modell.add(Dropout(0.1))
    
for i in range(0,2):
    modell.add(Conv1D(filters=768, kernel_size=5, padding='same', activation='relu'))  
    modell.add(BatchNormalization())
    modell.add(MaxPooling1D(pool_size=2))
    modell.add(Activation('relu'))
for i in range(0,2):
    modell.add(Conv1D(filters=1024, kernel_size=3, padding='same', activation='relu'))
    modell.add(BatchNormalization())
    modell.add(Activation('relu'))
    
#modell.add(Bidirectional(LSTM(512, dropout=0.2, recurrent_dropout=0.2)))
modell.add(Dense(1024, activation='relu'))
modell.add(Dense(512, activation='relu'))
modell.add(Dense(128, activation='relu'))
modell.add(Dense(2, activation='softmax'))
modell.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model Compiled")



Model Compiled


In [8]:
print("Fitting")
modell.fit(x_train, y_train, epochs=3, batch_size=32,verbose=1)
print("Saving")
modell.save('VDCNN.h5')
print("Model Saved")

Fitting
Epoch 1/1
    8/16640 [..............................] - ETA: 18:04:39 - loss: 0.6570 - acc: 0.7500

KeyboardInterrupt: 